In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib as plt
from tqdm import tqdm, trange
import torch
import pickle

In [2]:
#set parameters
vertices = 1000
adjacencies = 100
pi_vals = [0.25, 0.5, 0.75]
num_trials = 1000

In [3]:
replicate = {}

In [4]:
device = torch.device("cuda")

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [7]:
%ls

 12_9_23_Equation_1_RandomGraph_batch1.csv
 12_9_23_Equation_1_RandomGraph_batch1.pkl
 12_9_23_Equation_1_RandomGraph_batch2.pkl
 12_9_23_Equation_1_RandomGraph_batch3.pkl
 12_9_23_Equation_1_RandomGraph_batch4.pkl
'2023 lifting spreadsheet.gsheet'
'28 Training Programs (Excel).xlsx'
 academic_transcript.pdf
 ahskhghk.gdoc
'asb app.gdoc'
'AWA Printavo - #6240 - Bay Area Custom Shirts, Inc_.gdoc'
'AWA Printavo - #6240 - Bay Area Custom Shirts, Inc_.pdf'
'Belvedere Trading Recruiting Expense Form.xlsx'
'Copy of EIR Comment Letter Basic Template #1: Sargent Quarry Project .gdoc'
'Copy of Expense Tracker (final).gsheet'
'Copy of Flexible J&T 2.0 (1).gsheet'
'Copy of Flexible J&T 2.0.gsheet'
'Copy of GZCLP Spreadsheet v2.01 [Say No To Broscience].gsheet'
'Copy of Random Walk   Martingale Word Vomit.gdoc'
'Copy of SBS Strength Program reps to failure.gsheet'
'Copy of Sheiko Beginner Program | LiftVault.com.gsheet'
'Copy of The Rippler - Dr Workout.xlsx'
 CS229_Final_Project_Writeup.pdf
'hold

In [21]:
#beta 1 is treatment assignment
#beta 2 is neighbors
#beta 3 is second degree neighbors
def add_test_to_dictionary(beta_1, beta_2, beta_3, replicate):
  for pi in pi_vals:
    ADE = []
    AIE_1 = []
    AIE_2 = []
    AIE_3 = []
    AIE_4 = []
    for i in trange(1000):
      G = nx.random_regular_graph(adjacencies, vertices)

      #first degree neighbor matrix
      A = nx.adjacency_matrix(G).todense()

      #second degree neighbor matrix
      A_gpu = torch.tensor(A).float().to(device)
      A_2_gpu = torch.matmul(A_gpu, A_gpu)
      A_2 = A_2_gpu.cpu().numpy()
      A_2 = np.where(A_2 > 0, 1, 0)
      np.fill_diagonal(A_2, 0)
      A_2 = A_2 - A # second degree neighbors
      A_2_gpu = torch.tensor(A_2).float().to(device)

      #treatment assignment
      W = np.random.binomial(1, pi, vertices)
      W_gpu = torch.tensor(W).float().to(device)

      if beta_2 != 0:
        W_A = torch.matmul(W_gpu, A_gpu).cpu().numpy()
      else:
        W_A = 0

      if beta_3 != 0:
        W_A_2 = torch.matmul(W_gpu, A_2_gpu).cpu().numpy()
      else:
        W_A_2 = 0

      #noise
      noise = np.random.normal(0, 1, vertices)

      #generate Y
      Y = ((W_A_2)/np.sum(A_2, axis=1))*beta_3 + ((W_A)/np.sum(A, axis=1))*beta_2 + W * (beta_1) + noise

      #ADE
      average_direct_effect = ((np.dot(Y,W))/(pi) - (np.dot(Y, (1-W)))/(1-pi))/vertices
      ADE.append(average_direct_effect)

      #AIE
      #E = first degree
      average_indirect_effect1 = np.dot(Y, ((W_A)/pi - ((1-W)@A)/(1-pi)))/vertices
      average_indirect_effect2 = np.dot(Y, ((W_A_2)/pi - ((1-W)@A_2)/(1-pi)))/vertices
      #average_indirect_effect3 = np.dot(Y, ((W @ (A_2 + A))/pi - ((1-W)@ (A_2 + A))/(1-pi)))/vertices
      #average_indirect_effect4 = np.dot(Y, ((W @ A)/np.sum(W) - ((1-W)@A)/np.sum(1-W)))
      #O_W = np.multiply(O, np.array([W,]*O.shape[0]))
      #O_1_W = np.multiply(O, np.array([(1-W),]*O.shape[0]))
      #np.divide(a, b, out=np.zeros_like(a), where=b!=0)
      #M = (np.dot(Y, O_W))
      #B = np.sum(O_W, axis=0)
      #K = np.sum(O_1_W, axis=0)
      #P = np.dot(Y, O_1_W)
      #average_indirect_effect2 = np.sum(M)/np.sum(W) - np.sum(P)/np.sum(1-W)
      #average_indirect_effect1 = np.sum(np.divide(M, B, out=np.zeros_like(B, dtype=float), where=B!=0))/np.count_nonzero(B) - np.sum(np.divide(P, K, out=np.zeros_like(K, dtype=float), where=K!=0))/np.count_nonzero(K)
      #average_indirect_effect3 = (((np.dot(Y @ A, W))/(0.25) - (np.dot(Y @ A, (1-W)))/(0.75)))/(vertices)
      #average_indirect_effect4 = np.dot(Y, ((W @ A)/0.25 - ((1-W)@A)/(1-0.25)))/vertices
      #average_indirect_effect5 = np.dot(Y, ((W @ O)/0.25 - ((1-W)@O)/(1-0.25)))/vertices
      #average_indirect_effect5 = np.dot(Y, ((W @ O)/np.sum(W) - ((1-W)@O)/np.sum(1-W)))
      AIE_1.append(average_indirect_effect1)
      AIE_2.append(average_indirect_effect2)

    str1 = str(pi)+" ADE_{}_{}_{}".format(round(beta_1, 2), round(beta_2, 2), round(beta_3, 2))
    str2 = str(pi)+" AIE_1_{}_{}_{}".format(round(beta_1, 2), round(beta_2, 2), round(beta_3, 2))
    str3 = str(pi)+" AIE_2_{}_{}_{}".format(round(beta_1, 2), round(beta_2, 2), round(beta_3, 2))
    replicate[str1] = ADE
    replicate[str2] = AIE_1
    replicate[str3] = AIE_2

    with open('12_9_23_Equation_1_RandomGraph_batch4.pkl', 'wb') as fp:
      pickle.dump(replicate, fp)



In [22]:
betas = []
# baseline where no interference
#betas.append((1, 0, 0))

#only first degree neighbors
#betas.append((1/3, 2/3, 0))
#betas.append((0.5, 0.5, 0))
#betas.append((2/3, 1/3, 0))

#only second degree neighbors
#betas.append((1/3, 0, 2/3))
#betas.append((0.5, 0, 0.5))
#betas.append((2/3, 0, 1/3))

#first and second degree neighbors
#betas.append((1/3, 5/9, 1/9))
betas.append((1/3, 1/3, 1/3))
betas.append((1/3, 1/9, 5/9))

In [23]:
for beta in betas:
  add_test_to_dictionary(beta[0], beta[1], beta[2], replicate)

100%|██████████| 1000/1000 [07:21<00:00,  2.26it/s]


In [19]:
df_r = pd.DataFrame.from_dict(replicate)
df_r.head()

""


In [11]:
df_r.mean()

0.25 ADE_0.67_0.33_0        0.662160
0.25 AIE_1_0.67_0.33_0      0.258053
0.25 AIE_2_0.67_0.33_0   -222.650244
0.5 ADE_0.67_0.33_0         0.667727
0.5 AIE_1_0.67_0.33_0       0.249393
0.5 AIE_2_0.67_0.33_0    -449.785078
0.75 ADE_0.67_0.33_0        0.666975
0.75 AIE_1_0.67_0.33_0      0.311312
0.75 AIE_2_0.67_0.33_0   -675.344079
0.25 ADE_0.33_0_0.67        0.333756
0.25 AIE_1_0.33_0_0.67    -25.008550
0.25 AIE_2_0.33_0_0.67      1.723272
0.5 ADE_0.33_0_0.67         0.332744
0.5 AIE_1_0.33_0_0.67     -49.824180
0.5 AIE_2_0.33_0_0.67       1.019220
0.75 ADE_0.33_0_0.67        0.328593
0.75 AIE_1_0.33_0_0.67    -75.061363
0.75 AIE_2_0.33_0_0.67     -0.043270
dtype: float64

In [12]:
df_r.to_csv('12_9_23_Equation_1_RandomGraph_batch1.csv')